<a href="https://colab.research.google.com/github/wdempsey/AI4Health-Online-Experimentation/blob/main/part1_mrt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 1: Introduction to MRTs and JITAIS



In section 1, we will discuss the _micro-randomized trial design_ and the corresponding primary data analysis methods.  By the end of this section, you should be able to answer the following set of questions:
- What is a just-in-time adaptive intervention (JITAI)? 
- What is a micro-randomized trial?
- What is a causal excursion effect?  How does one estimate this effect?
- What are the main goals of early-stage optimization trials?

In addition, we will introduce the _HeartSteps simulator_.  This is a suite of functions that you can access via the GitHub Repository (in folder ``./hs_simulator/``) which allows us to generate synthetic users in a synthetic MRT for increasing physical activity.  We will use this simulator throughout this practical session. 

In [48]:
## Import packages and check code cells run
import numpy as np
import scipy as sp
from sklearn.linear_model import LinearRegression

# Part 1: Just-in-time adaptive interventions


A JITAI is ``is an intervention design aiming to provide the right type/amount of support, at the right time, by adapting to an individual's changing internal and contextual state'' (Nahum-Shani, 2018).  

- The term “just-in-time support” is used to describe an attempt to provide the right type (or amount) of support, at the right time.
- Timing is largely event-based, e.g., "a moment of high vulnerability and high receptivity".  
  - Ex. For a person attempting to quit smoking, a moment of high stress may lead to high likelihood of relapse.  If the person is currently available (e.g., no meeting on Google Calendar) and not currently active (e.g., not out for a walk), then the person may be receptive to a brief prompt aimed at reducing proximal stress.
  - This is not "timing-based", e.g., need to take pills at 4pm every day.



There are 5 key components of a JITAI:
- __Decision Points__: A decision point is a time at which an intervention decision is made. 
- __Intervention Options__: An array of possible treatments or actions that might be employed at any given decision point.
- __Tailoring Variable__: A tailoring variable is information concerning the individual that is used to decide when (i.e., under what conditions) to provide an intervention and which intervention to provide. 
- __Outcome__:
  - __Distal outcome__: Ultimate goal the intervention is intended to achieve; it is usually a primary clinical outcome, such as weight loss, drug/alcohol use reduction or increase in average activity level.  
  - __Proximal outcome__: Short-term goals the intervention options are intended to achieve.  Typically thought to be on the causal pathway (i.e., a mediator).
- __Decision rules__:  Operationalize the adaptation by specifying which intervention option to offer, for whom, and when. 


A JITAI is an _intervention design_.  Behavioral scientists often have questions in how to best design a JITAI for a particular behavioral health setting.  Consider an mHealth smoking cessation setting.  Scientists may wish to intervene by either sending a reminder to practice mindfulness (hopefully reducing proximal stress) or not; however, it is unknown whether sending the message when the individual is currently stressed (high vulnerability but low receptivity) is better than when the individual is current not stressed (low vulnerability but high receptivity).  

We will break for 10 minutes for small group intros and a short exercise.

- __Group Task 1__: Identify the 5 elements from the following decision rule in a recovery support services mHealth study (A-CHESS):
  - __``If``__ ``At High Risk Location``, __``Then``__ ``IO = Send Message``, __``Else``__ ``IO = Do Nothing``.
- __Group Task 2__: Construct a JITAI to be included in a smoking cessation mHealth intervention package aimed at reducing proximal stress.  Be sure to highlight the 5 key elements.   


#Part 2: Micro-randomized trials (MRTs)

MRTs are an experimental design to collect data to answer questions about the construction of JITAIs. 

- For each person in a study, let $t=1,\ldots, T$ denote a sequence of decision points.  
- At each decision time $t$,  we observe a state variable $S_t \in \mathbb{R}^p$.  
- After observing the state variable $S_t$, the _clinical trialist_ decides to take action $A_t \in \mathcal{A}$ with probability $p_t (A_t \mid H_t)$ (i.e., the randomization probability may depend on the observed history $H_t$).  
- After observing state $S_t$ and taking action $A_t$, the agent observes the proximal response $Y_{t+1}$.  The proximal response is a deterministic function of state, action, and next state (i.e., $Y_{t+1} = g(S_t, A_t, S_{t+1})$)
- The sequence of state, action, and reward at a sequence of decision points defines a _micro-randomized trial_, $\{ S_t, A_t, Y_{t+1} \}_{t=1}^T$.
- Here, our goal is to collect data to optimize an intervention component
  - Q1: Should we include this intervention component in an overall intervention package?
  - Q2: What should the decision rule be in the optimized JITAI?

## Part 2a: A simple MRT example ($n=1$)

- $T = 200$
- $S_t = (S_{t1}, S_{t2})$ where $S_{t1}$ is continuous and $S_{t2}$ is a binary state
- $A_t \sim \text{Bern}(0.6)$
- Define the proximal outcome as
$$
Y_{t+1} = S_t^\prime \alpha + (A_t - 0.6) S_t^\prime \beta 
$$

In [55]:
## What about if the probabilities are non-constant?
mrt_params = np.array([0.,1.,-np.log(1/0.7-1)])

n = 500
study_ids, study_states, study_probs, study_actions, study_outcomes = generate_mrt(n, state_params, mrt_params, T)
print("Action probability in state2 = 0 & 1")
print(np.unique(study_probs))

## Build the design matrix
X = np.concatenate(study_states, axis=0 )
actions = np.concatenate(study_actions, axis = 0)
probs = np.concatenate(study_probs, axis = 0)
outcomes = np.concatenate(study_outcomes, axis = 0)

for col in range(2):
  temp = np.multiply(X[:,col],actions)
  X = np.column_stack((X, temp))

reg = LinearRegression().fit(X,outcomes)
print("True coefficients using linear model")
print(np.array([1,0.3,0.5,-0.7]))
print("Fitted coefficients using linear model")
print(reg.coef_)

X2 = X[:,(0,2,3)]

reg = LinearRegression().fit(X2,outcomes)
print("Fitted coefficients using misspecified linear model")
print(reg.coef_)

X = np.concatenate(study_states, axis=0 )

for col in range(2):
  temp = np.multiply(X[:,col],actions-probs)
  X = np.column_stack((X, temp))

reg = LinearRegression().fit(X,outcomes)
print("Fitted coefficients using action-centered linear model")
print(reg.coef_)

X2 = X[:,(0,2,3)]

reg = LinearRegression().fit(X2,outcomes)
print("Fitted coefficients using misspecified, action-centered linear model")
print(reg.coef_)

# Part 3a: Causal Excursion Effects

We define the _causal excursion effect_ by

$$
\beta(t) = \mathbb{E} \left[ Y_{t+1}(\bar A_{t-1}, 1) - Y_{t+1} (\bar A_{t-1}, 0) \right]
$$

It is similar to the ATE (i.e., averaging over the covariate distribution); however, in our current setting, we are also average over __prior treatments__.  Thus the effect is a single decision point _excursion_ from the MRT randomization probability. Under the following (standard) causal inference assumptions,

- **Positivity**:  Treatment probability is bounded away from 0 and 1.
- **Sequential ignorability**:  potential outcomes are independent of $A_t$ given the observed history $H_t$.
- **Consistency**:  The observed values are equal the corresponding potential outcomes, i.e., $Y_{t+1} (\bar A_t) = Y_{t+1}$.  

the effect can be re-expressed as

$$
\beta(t) = \mathbb{E} \left[ Y_{t+1} \mid A_t = 1 \right] - \mathbb{E} \left[ Y_{t+1} \mid A_t = 0 \right].
$$

Note, that $\beta(t)$ is a treatment effect that may vary __over time__!


### Moderated effect

Often, we want to understand if certain time-varying covariates _moderate_ the treatment effect.  That is, does the effect of treatment change given the individual is in a particular state.  To address this, we define the 

$$
\beta(t;s) = \mathbb{E} \left[ Y_{t+1}(\bar A_{t-1}, 1) - Y_{t+1} (\bar A_{t-1}, 0) \mid S_t (A_t) = s \right]
$$

- This is causal excursion effect is conditional on a summary variable being at value $s$.  
- The summary variable may be affected by prior treatment.  
- It is similar to the conditional ATE (i.e., averaging over the covariate distribution except for a particular covariate); 
- In our current setting, we again are also averaging over __prior treatments__ unless the summary variables contain it (i.e., $A_{s} (\bar A_{s-1}) \subset S_t (\bar A_{t-1})$).
- The effect can be expressed in terms of observable data as

$$
\beta(t;s) = \mathbb{E} \left[  \mathbb{E} \left[ Y_{t+1} \mid A_t = 1, H_t \right] - \mathbb{E} \left[ Y_{t+1} \mid A_t = 0, H_t  \right] \mid S_t = s \right]
$$

- __Question 1__: Why may we not want to consider the fully conditional effect?
- __Question 2__: What are some moderators in the simple simulation study?


# Part 3b: Primary analysis method

A simple proposal for estimating the marginal causal excursion effect is to simply take the average of all individuals at each time
$$
\frac{\sum_{j=1}^n Y_{t+1} A_t}{\sum_{j=1}^n A_t} - 
\frac{\sum_{j=1}^n Y_{t+1} (1-A_t)}{\sum_{j=1}^n (1-A_t)} 
$$

- When is this OK?  When may this be problematic?

The issue is that treatment assignment may not be completely at random.  If the action probability depends on the observed history, then some individuals are more likely to receive treatment at certain times than others. 

- One solution is __inverse weighting__ 

\begin{align*}
\mathbb{E} \left[ Y_{t+1} \frac{1[A_t = 1]}{p_t(A_t \mid H_t)} \right] 
&= 
\mathbb{E} \left[ \mathbb{E} \left[ Y_{t+1} \frac{1[A_t = 1]}{p_t(A_t \mid H_t)} \mid H_t \right] \right] \\
&= \mathbb{E} \left[ \mathbb{E} \left[ Y_{t+1}  \mid H_t, A_t = 1 \right] \right] \\
\end{align*}

- Here, $W_t = 1[A_t = 1]/ p_t (A_t \mid H_t)$ is the _inverse-probability weight_.


## Putting together weighting and action-centering

Now that we know about action-centering and weighting, we can define the __weighted-centered least squares__ criterion used to estimate 

$$
\mathbb{P}_n \left[ \sum_{t=1}^T \frac{\tilde p_t (A_t \mid S_t)}{p_t (A_t \mid H_t)} \left( Y_{t+1} - g_t (H_t)^\prime \alpha - (A_t - \tilde p_t (1 \mid S_t)) f_t (S_t)^\prime \beta \right)^2  \right]
$$

-where $\mathbb{P}_n$ just signifies
- Note that the model 




# Part 3c: Simple simulation practice

__Group exercise__ (20 minutes): 

- Use the simulated MRT data to estimate the time-varying treatment and moderated treatment effect.
- Plot the effect as a function of day in study.
- Show that weights are necessary to estimate the marginal causal excursion effect.
- Show that weights are not necessary to estimate the causal excursion effect conditional on current state.
- What do the results tell us about the intervention component? 

In [69]:
## Fitting the WCLS

tilde_p = np.mean(probs)*actions + (1-np.mean(probs)) * (1-actions)
weight = tilde_p / probs

## Build the design matrix
actions = np.concatenate(study_actions, axis = 0)
probs = np.concatenate(study_probs, axis = 0)
outcomes = np.concatenate(study_outcomes, axis = 0)

X = actions - probs
X = X[:,np.newaxis]

## The unweighted model
reg = LinearRegression().fit(X,outcomes)
print("True coefficients using linear model")
print(np.array([-0.7*0.7]))
print("Fitted coefficients using unweighted LS")
print(reg.coef_)

## The weighted model
reg_weight = LinearRegression().fit(X,outcomes, weight)
print("Fitted coefficients using weighted LS")
print(reg_weight.coef_)


True coefficients using linear model
[-0.49]
Fitted coefficients using unweighted LS
[-0.26031311]
Fitted coefficients using weighted LS
[-0.27516706]
